In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

sys.path.insert(0, "..")
from utils import load_all, get_dataset_names, scheirer_ray_hare_test, calc_cliffs_d
from utils import RESULTSPATH, CMAP, SEGMENTS
from overall_comparision.general_analyses import calc_DI_lengths

plt.style.use("seaborn")

## Load data and preprocessing

In [2]:
# in vitro against in vivo datasets
vitro_dfnames = get_dataset_names(cutoff=40, selection="in vitro")
vitro_dfs, _ = load_all(vitro_dfnames)
vivo_dfnames = get_dataset_names(cutoff=40, selection="in vivo mouse")
vivo_dfs, _ = load_all(vivo_dfnames)
human_dfnames = get_dataset_names(cutoff=40, selection="in vivo human")
human_dfs, _ = load_all(human_dfnames)

In [3]:
IAV_dfnames = get_dataset_names(cutoff=40, selection="IAV")
IBV_dfnames = get_dataset_names(cutoff=40, selection="IBV")

vitro_dfnames = get_dataset_names(cutoff=40, selection="in vitro")
vivo_dfnames = get_dataset_names(cutoff=40, selection="in vivo mouse")    
human_dfnames = get_dataset_names(cutoff=40, selection="in vivo human")

def get_IV_type(datasetname):
    if datasetname in IAV_dfnames:
        return "IAV"
    elif datasetname in IBV_dfnames:
        return "IBV"
    else:
        return "error"
    
def get_host_system(datasetname):
    if datasetname in vitro_dfnames:
        return "in vitro"
    elif datasetname in vivo_dfnames:
        return "in vivo mouse"
    elif datasetname in human_dfnames:
        return "in vivo human"
    else:
        return "error"

### Create subplots for figure 2

In [ ]:
def compare_DI_lengths(dfs: list, dfnames: list, labels: str, analysis: str="")-> None:
    '''
        compares the lengths of the DelVGs between three classes.
        :param dfs: list of datasets
        :param dfnames: list of dataset names
        :param label: label for the datasets
        :param analysis: string to define the performed analysis.
                         needed for figure saving

        :return: None
    '''
    def process_data(dfs, dfnames):
        lengths_dict = calc_DI_lengths(dfs, dfnames)
        final_d = dict({"PB2": Counter(), "PB1": Counter(), "PA": Counter(), "HA": Counter(), "NP": Counter(), "NA": Counter(), "M": Counter(), "NS": Counter()})
        for d in lengths_dict.values():
            for s in d.keys():
                final_d[s] += Counter(d[s])
        return final_d

    def calc_stats(x_1, x_2, s, e, h):
        cliffs_d = calc_cliffs_d(x_1, x_2)
        plt.plot([s, e], [h, h], lw=1, color="black")
        plt.plot([s, s], [h, h+0.0002], lw=1, color="black")
        plt.plot([e, e], [h, h+0.0002], lw=1, color="black")
        if s == 800 and e == 1900:
            text_loc = (s+e+100)/2
        else:
            text_loc = (s+e)/2
        plt.text(text_loc, h-0.0007, f"{cliffs_d:.2f}", ha="center", va="bottom", color="black")
        return
    
    dicts = [process_data(df, name) for df, name in zip(dfs, dfnames)]
    figsize = (6, 2) if analysis == "vivo_vitro" else (5, 2)
    cm = plt.get_cmap(CMAP)
    colors = [cm(0/8), cm(3/8), cm(1/8)]
    bins = 30
    for s in ["PB2", "PB1", "PA"]:
        lists = [[k for k, v in d[s].items() for _ in range(v)] for d in dicts]
        max_p = max(max(l) for l in lists)
        skip = False
        for l in lists:
            if len(l) < 1:
                skip = True
        if skip == True:
            continue
        
        plt.figure(figsize=figsize, tight_layout=True)
        for i, l in enumerate(lists):
            plt.hist(l, alpha=0.5, label=labels[i], bins=bins, range=(0, max_p), density=True, color=colors[i])

        if len(lists) == 3:
            print(s)
            calc_stats(lists[0], lists[1], 800, 1300, 0.0025)
            calc_stats(lists[0], lists[2], 800, 1900, 0.0031)
            calc_stats(lists[1], lists[2], 1300, 1900, 0.0038)
        
        plt.ylim(0, 0.005)
        plt.xlim(0, 2500)
        plt.yticks([0, 0.0025, 0.005])
        plt.xticks([0, 500, 1000, 1500, 2000, 2500])
        plt.xlabel(f"DelVG sequence length for {s} (nt)")
        plt.ylabel("Probability density         ")
        plt.legend(loc="upper center", ncol=3)

        save_path = os.path.join(RESULTSPATH, "datasplits")
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        plt.tight_layout()
        plt.savefig(os.path.join(save_path, f"{s}_{analysis}.png"), dpi=300)
        plt.close()

labels = ["in vitro", "mouse", "human"]
compare_DI_lengths([vitro_dfs, vivo_dfs, human_dfs], [vitro_dfnames, vivo_dfnames, human_dfnames], labels, analysis="vivo_vitro")

PB2
U:	1250954.0
	3.018809726447335e-33
U:	939684.5
	9.143481652041818e-125
U:	584813.0
	5.139704486495526e-27
PB1
U:	4358330.5
	4.2476833314993497e-47
U:	2215338.5
	1.0676095607719163e-124
U:	1541022.0
	3.8007096887894955e-49
PA
U:	3624901.0
	2.173110654483113e-36
U:	1886535.5
	1.3993510860120682e-59
U:	1754845.0
	1.879239982011875e-07


## SRH test

In [5]:
lengths_dict = calc_DI_lengths(vitro_dfs + vivo_dfs + human_dfs, vitro_dfnames + vivo_dfnames + human_dfnames)

In [6]:
data = dict({s: pd.DataFrame({"Measure": list(),"IV_type": list(),"Host_system": list() }) for s in SEGMENTS})
data["Pooled"] = pd.DataFrame({"Measure": list(),"IV_type": list(),"Host_system": list() })

seg_counts = dict({s: 0 for s in SEGMENTS})

for dfname in vitro_dfnames + vivo_dfnames + human_dfnames:
    for seg in SEGMENTS:
        values_lengths = lengths_dict[dfname][seg]
        total_sum = sum(key * value for key, value in values_lengths.items())
        total_count = sum(values_lengths.values())
        if total_count > 0:
            mean = total_sum / total_count
        else:
            mean = 0
        
        seg_counts[seg] += total_count
        
        temp_data = pd.DataFrame({
            "Measure": [mean],
            "IV_type": [get_IV_type(dfname)],
            "Host_system": [get_host_system(dfname)]
        })
        data[seg] = pd.concat([data[seg], temp_data], ignore_index=True)

data["Pooled"] = pd.DataFrame({"Measure": list([0]*20),"IV_type": data["PB1"]["IV_type"],"Host_system": list(data["PB1"]["Host_system"]) })

for s in SEGMENTS:
    data["Pooled"]["Measure"] += data[s]["Measure"] * seg_counts[s]

data["Pooled"]["Measure"] /= sum(seg_counts.values())

In [8]:
# filter out non-poly segments and in vivo mouse data
for seg in ["Pooled"] + SEGMENTS:
    data[seg] = data[seg][data[seg]["Host_system"] != "in vivo mouse"].reset_index(drop=True)

### Comparison of the length means for in vitro/human and IAV/IBV

In [21]:
temp = data["Pooled"]

mean_vitro = temp[temp["Host_system"] == "in vitro"]["Measure"].mean()
mean_human = temp[temp["Host_system"] == "in vivo human"]["Measure"].mean()
print(mean_human - mean_vitro)

mean_iav = temp[temp["IV_type"] == "IAV"]["Measure"].mean()
mean_ibv = temp[temp["IV_type"] == "IBV"]["Measure"].mean()
print(mean_ibv - mean_iav)

232.46842257664355
120.60065363894739


### Perform Scheirer-Ray-Hare test

In [9]:
p_iv_list = list()
p_host_list = list()
p_inter_list = list()
for seg in ["Pooled", "PB2", "PB1", "PA"]:
    print(seg)
    if seg == "Pooled":
        H_iv, p_iv, H_host, p_host, H_int, p_int = scheirer_ray_hare_test(data[seg])
    else:
        H_iv, p_iv, H_host, p_host, H_int, p_int = scheirer_ray_hare_test(data[seg])
    print("IV type host  interaction")
    print(round(H_iv,2), round(H_host,2), round(H_int,2))
    print(round(p_iv,4), round(p_host,4), round(p_int,4))
    print()
    p_iv_list.append(p_iv)
    p_host_list.append(p_host)
    p_inter_list.append(p_int)

Pooled
IV type host  interaction
1.93 6.91 7.16
0.1651 0.0086 0.0074

PB2
IV type host  interaction
3.55 7.47 4.98
0.0596 0.0063 0.0257

PB1
IV type host  interaction
0.8 4.88 10.33
0.3722 0.0272 0.0013

PA
IV type host  interaction
3.18 9.3 3.52
0.0743 0.0023 0.0608



## Supplement Figure 

In [10]:
# in vitro against in vivo human
dfs = [vitro_dfs, human_dfs]
dfnames = [vitro_dfnames, human_dfnames]
labels = ["in vitro", "human"]
compare_DI_lengths(dfs, dfnames, labels, analysis="vivo_vitrohuman")

# all IAV against all IBV datasets
IAV_dfnames = get_dataset_names(cutoff=40, selection="IAV")
IAV_dfs, _ = load_all(IAV_dfnames)
IBV_dfnames = get_dataset_names(cutoff=40, selection="IBV")
IBV_dfs, _ = load_all(IBV_dfnames)

dfs = [IAV_dfs, IBV_dfs]
dfnames = [IAV_dfnames, IBV_dfnames]
labels = ["IAV", "IBV"]
compare_DI_lengths(dfs, dfnames, labels, analysis="IAV_IBV")

# in vitro all IAV against BLEE and Sheng
vitro_iav_dfs = list()
vitro_iav_dfnames = list()
vitro_ibv_dfs = list()
vitro_ibv_dfnames = list()
for df, dfname in zip(vitro_dfs, vitro_dfnames):
    if dfname in ["Alnaji2019_BLEE", "Sheng2018"]:
        vitro_ibv_dfs.append(df)
        vitro_ibv_dfnames.append(dfname)
    else:
        vitro_iav_dfs.append(df)
        vitro_iav_dfnames.append(dfname)

dfs = [vitro_iav_dfs, vitro_ibv_dfs]
dfnames = [vitro_iav_dfnames, vitro_ibv_dfnames]
labels = ["IAV in vitro", "IBV in vitro"]
compare_DI_lengths(dfs, dfnames, labels, analysis="vitro_IAV")

# in vivo human all IBV against Berry A
vivo_iav_dfs = list()
vivo_iav_dfnames = list()
vivo_ibv_dfs = list()
vivo_ibv_dfnames = list()
for df, dfname in zip(human_dfs, human_dfnames):
    if dfname == "Berry2021_A":
        vivo_iav_dfs.append(df)
        vivo_iav_dfnames.append(dfname)
    else:
        vivo_ibv_dfs.append(df)
        vivo_ibv_dfnames.append(dfname)

dfs = [vivo_iav_dfs, vivo_ibv_dfs]
dfnames = [vivo_iav_dfnames, vivo_ibv_dfnames]
labels = ["IAV human", "IBV human"]
compare_DI_lengths(dfs, dfnames, labels, analysis="vivo_IBV")

# IAV vitro vs vivo human
dfs = [vitro_iav_dfs, vivo_iav_dfs]
dfnames = [vitro_iav_dfnames, vitro_ibv_dfnames]
labels = ["IAV in vitro", "IAV human"]
compare_DI_lengths(dfs, dfnames, labels, analysis="IAV_vitro_vivo")

# IBV vitro vs vivo human
dfs = [vitro_ibv_dfs, vivo_ibv_dfs]
dfnames = [vivo_iav_dfnames, vivo_ibv_dfnames]
labels = ["IBV in vitro", "IBV human"]
compare_DI_lengths(dfs, dfnames ,labels, analysis="IBV_vitro_vivo")